In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import time

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
start = time.time()

driver = webdriver.Chrome()
driver.get("https://www.cgs.or.kr/business/esg_tab04.jsp")

time.sleep(2)  # 페이지 로딩 대기

#5. 등급조회 버튼 클릭
search_btn = driver.find_element("xpath", '//*[@id="listPanel"]/ul[2]/li[1]/div/div[2]/table/tbody/tr/td/div/button')
search_btn.click()

#6. 등급조회에서 '학생 및 학술연구자' 항목 클릭
usertype = driver.find_element("xpath", '//*[@id="usertype6"]')
driver.execute_script("arguments[0].click();", usertype)
time.sleep(1)

#7. 등급조회에서 '동의' 클릭
usertype = driver.find_element("xpath", '//*[@id="popupGradeInquiry"]/div/div/div[2]/div/div/label[1]/div/input')
driver.execute_script("arguments[0].click();", usertype)
time.sleep(1)

#8. 선택 버튼 클릭
search_btn = driver.find_element("xpath", '//*[@id="popupGradeInquiry"]/div/div/button')
search_btn.click()

time.sleep(2)

# 1. 드롭다운 <select> 요소 찾기 (id가 sfyear)
select_element = driver.find_element("id", "sfyear")

# 2. Select 객체로 래핑
select = Select(select_element)

# 3. 원하는 연도로 선택 변경 (예: "2023")
select.select_by_value("2023")

# 4. 연도 선택 후 데이터 새로 로딩되도록 대기
time.sleep(2)  # 로딩 대기

#5. 조회 버튼 클릭하기
search_btn = driver.find_element("xpath", '//*[@id="searchform"]/button')
search_btn.click()

#================이제 여기까지 2023~2024년도 조회까지는 완료됨=====================#

In [ ]:
data_list = []
page_num = 0
total_company_num = 0
while(True):
    for i in range(1, 11):  # 예시: 1~11까지 루프
        page_num += 1
        print(f"================={page_num} 번째 페이지 크롤링 시작===============")
        try:
            page_btn = driver.find_element(By.XPATH, f'//*[@id="listPanel"]/ul[2]/li[1]/div/div[3]/span/a[{i}]')
            page_btn.click()
            time.sleep(1.5)

            # 테이블 크롤링
            rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")
            for row in rows:
                cols = row.find_elements(By.TAG_NAME, "td")
                if len(cols) < 9: # 전체 열 수수
                    continue  # 설명 테이블 or 공백
                data = [col.text.strip() for col in cols]
                print(data)
                data_list.append(data)
                total_company_num += 1
        except:
            continue  # 페이지 버튼 없을 수도 있음

    try:
        # 다음 블록이 있을 경우 "다음" 클릭
        next_block = driver.find_element(By.XPATH, '//*[@id="listPanel"]/ul[2]/li[1]/div/div[3]/button[2]')
        # 마지막 페이지에 도달한 경우
        onclick_attr = next_block.get_attribute("onclick")
        if onclick_attr is None or onclick_attr.strip() == "":
            print("마지막 페이지 블록 도달, 루프 종료")
            break
            
        next_block.click()
        time.sleep(2)
    except:
        print("다음 버튼을 찾을 수 없음 – 종료")
        break
print("총 기업 수: ", total_company_num)

ent = time.time()
print("크롤링 하는 데 걸린 총 시간: ", end - start)

In [ ]:
import pandas as pd

# 크롤링 종료 후 저장
columns = ['NO', '기업명', '기업코드', 'ESG등급', '환경', '사회', '지배구조', '년도', '첨부']
df = pd.DataFrame(data_list, columns=columns)

# CSV로 저장
df.to_csv("esg_data.csv", index=False, encoding="utf-8-sig")
print("CSV 파일 저장 완료: esg_data.csv")